In [2]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder



In [3]:
import pandas as pd
from category_encoders import BinaryEncoder

data = pd.DataFrame({'color': ['red', 'green', 'blue', 'red']})

encoder = BinaryEncoder(cols=['color'])
encoded_data = encoder.fit_transform(data)

print(encoded_data)

   color_0  color_1
0        0        1
1        1        0
2        1        1
3        0        1


/Users/muralis/AIML/deepl/lib/python3.12/site-packages/sklearn/base.py:411: FutureWarning: The `_get_tags` method is deprecated in 1.6 and will be removed in 1.7. Please implement the `__sklearn_tags__` method.
  warnings.warn(


In [4]:
data =file = pd.read_csv("/users/muralis/AIML/data/Churn_Modelling.csv") 
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## preprocess data 
## Drop irrelevant data
data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
##Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [7]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geo = OneHotEncoder()
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [8]:
one_hot_encoder_geo.get_feature_names_out()
print(geo_encoder.toarray())

geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=one_hot_encoder_geo.get_feature_names_out())

[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [9]:
##combine one hot encoded columns with the original data
data = pd.concat([data.drop('Geography', axis=1),geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
##Save the encoder and scaler
with open('label_encoder_gender.pk1', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open ('one_hot_encoder_geo.pk1', 'wb') as file:
    pickle.dump(one_hot_encoder_geo,file)


In [11]:
## divide the data into independent and dependent data set
x= data.drop('Exited', axis=1)
y= data['Exited']
print (x)
print(y)

      CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0             619       0   42       2       0.00              1          1   
1             608       0   41       1   83807.86              1          0   
2             502       0   42       8  159660.80              3          1   
3             699       0   39       1       0.00              2          0   
4             850       0   43       2  125510.82              1          1   
...           ...     ...  ...     ...        ...            ...        ...   
9995          771       1   39       5       0.00              2          1   
9996          516       1   35      10   57369.61              1          1   
9997          709       0   36       7       0.00              1          0   
9998          772       1   42       3   75075.31              2          1   
9999          792       0   28       4  130142.79              1          1   

      IsActiveMember  EstimatedSalary  Geography_Fr

In [27]:
## split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.33, random_state=42)

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test= scaler.fit_transform(x_test)

In [28]:
##Save scaler file
with open ('scalar.pk1','wb') as file:
    pickle.dump(scaler, file)

In [29]:
x_train.shape[1]

12

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model =Sequential([ Dense(64, activation='relu', input_shape=(x_train.shape[1],),),
            Dense(32,activation='relu'),
            Dense(1,activation='sigmoid')
                ])
model.summary()

/Users/muralis/AIML/deepl/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
## define the optimizer and loss function and then compile the modela
##  the compile needs to know the optimizer function, the loss function and what are we optimizing for

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

opt = Adam(learning_rate=0.01)
loss = BinaryCrossentropy()

model.compile(opt,loss,metrics=['accuracy'])

In [32]:
## now define the call back functionA
## Earlystopping allows the epoch to stop if you do not see progress further 
## TensorBoard will help you to 

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard



In [33]:
## you define the call bacl function and the early stop function and will use it for the training
import datetime
logdir = '/Users/muralis/AIML/data/log/fit/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = TensorBoard(log_dir=logdir,histogram_freq=1)

earlystop =EarlyStopping(monitor='val_loss',patience=10, restore_best_weights=True)


In [34]:
## train the model

from tensorflow.keras.saving import save_model
hist = model.fit(x=x_train, y=y_train, epochs=100,validation_data=(x_test,y_test), callbacks=[tensorboard_cb,earlystop])
model.save('/Users/muralis/AIML/data/model.h5')
save_model(model,'/Users/muralis/AIML/data/mymodel.keras')

%load_ext tensorboard
%tensorboard --logdir /Users/muralis/AIML/data/log/fit



Epoch 1/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 874us/step - accuracy: 0.8030 - loss: 0.4562 - val_accuracy: 0.8615 - val_loss: 0.3536
Epoch 2/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - accuracy: 0.8522 - loss: 0.3591 - val_accuracy: 0.8642 - val_loss: 0.3437
Epoch 3/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - accuracy: 0.8571 - loss: 0.3392 - val_accuracy: 0.8509 - val_loss: 0.3694
Epoch 4/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 606us/step - accuracy: 0.8504 - loss: 0.3572 - val_accuracy: 0.8633 - val_loss: 0.3499
Epoch 5/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - accuracy: 0.8627 - loss: 0.3335 - val_accuracy: 0.8582 - val_loss: 0.3491
Epoch 6/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 580us/step - accuracy: 0.8613 - loss: 0.3307 - val_accuracy: 0.8652 - val_loss: 0.3350
Epoch 7/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step - accuracy: 0.8692 - loss: 0.3359 - val_accuracy: 0.8627 - val_loss: 0.3389
Epoch 8/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 569us/step - accuracy: 0.8633 - loss: 0

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 94270), started 0:06:56 ago. (Use '!kill 94270' to kill it.)

In [35]:
%load_ext tensorboard
%tensorboard --logdir /Users/muralis/AIML/data/log/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 94270), started 0:07:48 ago. (Use '!kill 94270' to kill it.)

In [25]:
 %reload_ext tensorboard